This notebook will be used to compare currents of the Salish Sea Model to Mike Foreman's Model using tidal ellipses.

In [19]:
import os
import glob
import datetime
import matlab.engine
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import datetime
import pandas as pd

import netCDF4 as nc
from scipy.optimize import curve_fit
import scipy.io as sio
from salishsea_tools import (viz_tools, tidetools, nc_tools)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline

ImportError: No module named matlab.engine

In [2]:
M2 = sio.loadmat('/ocean/mdunn/Tides/vitides/M2.mat')

In [29]:
print M2['Major'].shape
raddeg = 180/np.pi
print raddeg

(67824, 1)
57.2957795131


In [81]:
def gcdist_py(lat1,lon1,lat2,lon2):
    """Function to calculate distance in kilometers and heading between two positions in latitude and longitude.
    Assumes -90 > lat > 90  and  -180 > long > 180
    north and east are positive
    Uses law of cosines in spherical coordinates to calculate distance
    calculate conversion constants"""

    raddeg = 180/np.pi
    degrad=1/raddeg

    # convert latitude and longitude to radians

    lat1=lat1*degrad
    lat2=lat2*degrad

    in1=np.where(lon1>180)
    print lon1
    lon1[in1] = lon1[in1]-360

    in2=find(lon2>180)

    lon2[in2] = lon2[in2]-360;

    lon1=-lon1*degrad;
    lon2=-lon2*degrad;
    # calculate some basic functions
    coslat1=np.cos(lat1);
    sinlat1=np.sin(lat1);
    coslat2=np.cos(lat2);
    sinlat2=np.sin(lat2);
    #calculate distance on unit sphere
    dtmp=np.cos(lon1-lon2);
    dtmp=(sinlat1*sinlat2) + (coslat1*coslat2*dtmp)

    # check for invalid values due to roundoff errors
    in1=find(dtmp>1.0)
    dtmp[in1]=1.0;
    in2=find(dtmp<-1.0)
    dtmp[in2]=-1.0;

    # convert to meters for earth distance
    ad = acos(dtmp);
    # radius of Earth at Equator: 40075km, Pole to pole: 40008
    # average km per degree latitude = 111.2263888
    #d=(111.112) .* raddeg .* ad;
    d=(111.2263888) * raddeg * ad

    # now find heading
    hdgcos = (sinlat2-sinlat1n*np.cos(ad))/(np.sin(ad)*coslat1);

    # check value to be legal range
    in1=find(hdgcos>1.0)
    hdgcos[in1]=1.0;
    in2=find(hdgcos<-1.0)
    hdgcos[in2]=-1.0
    hdg = acos(hdgcos)*raddeg;

    # if longitude is decreasing then heading is between 180 and 360
    test = sin(lon2-lon1);
    in1=find(test>0.0);
    hdg[in1]=360-hdg(in1);

    return d, hdg

In [82]:
def vitide_py(lon,lat,times):

    sio.loadmat('/ocean/mdunn/Tides/vitides/NewVIGF8.mat')

    d=np.ones((67824,1))


    for i in np.arange(0,67823): 
        d[i]=gcdist_py(lat,long,Lat[i],Long[i])

    [D,I]=sort(d);
    NN=3; # how many close nodes (3-5)
    I=I[1:NN]; # these are the NN clostest nodes.
    D=D[1:NN];
    idwa=sum(D^-1);
    
    sio.loadmat('/ocean/mdunn/Tides/vitides/viconst.mat')   # load vicon and F
    sio.loadmat('/ocean/mdunn/Tides/vitides/vitidecon.mat') # load the fmajor, fminor, etc. matrices
    disp(' Calculating the tides at the closests nodes.');
    for i in np.arange(1,NN):
        j=I[i];
        tidecon=[fmaj[j,:],fmin[j,:],finc[j,:],pha[j,:]];
        UV=t_predic(times,vicon,F,tidecon);
        U[i,:]=real[UV]
        V[i,:]=imag[UV]
    end
    u=0;v=0;
    for i in np.arange(1,NN):
        u = u + U[i,:]/D[i];
        v = v + V[i,:]/D[i];
    end
    u=u/idwa;
    v=v/idwa;

    return u, v

In [87]:
to = datetime.datetime(2014,11,26)
tf = datetime.datetime(2015, 6, 21)

grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
Lat = grid_B.variables['nav_lat'][:]
Long = grid_B.variables['nav_lon'][:]
bathy = grid_B.variables['Bathymetry'][:,:]
print Long[1]
u, v = vitide_py(48.537579, -123.225789, np.arange(1,12))


[-123.43196869 -123.42677307 -123.42152405 -123.41625214 -123.41096497
 -123.4056778  -123.400383   -123.39508057 -123.38977814 -123.38447571
 -123.37917328 -123.37387085 -123.36856079 -123.36325073 -123.3579483
 -123.35263824 -123.34732819 -123.34201813 -123.33670807 -123.33139801
 -123.32608032 -123.32077026 -123.31546021 -123.31015015 -123.30484009
 -123.2995224  -123.29421234 -123.28890228 -123.28358459 -123.27827454
 -123.27295685 -123.26764679 -123.2623291  -123.25701904 -123.25170135
 -123.2463913  -123.24107361 -123.23576355 -123.23044586 -123.2251358
 -123.21981812 -123.21450806 -123.20919037 -123.20387268 -123.19856262
 -123.19324493 -123.18793488 -123.18261719 -123.1772995  -123.17198944
 -123.16667175 -123.16135406 -123.15604401 -123.15072632 -123.14540863
 -123.14009857 -123.13478088 -123.1294632  -123.12414551 -123.11883545
 -123.11351776 -123.10820007 -123.10289001 -123.09757233 -123.09225464
 -123.08693695 -123.08162689 -123.0763092  -123.07099152 -123.06567383
 -123.06

AttributeError: type object 'long' has no attribute 'shape'